# Script: Tucarro.com

This script allows you to follow up the Mercadolibre daily publications and identify variations in the price of products, in this case, cars publications.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

# Date Identifiers
today_file = str(datetime.today().strftime('%Y-%m-%d'))
yesterday_file = datetime.today()-timedelta(days=1)
yesterday_file = str(yesterday_file.strftime('%Y-%m-%d'))

# Path where the files will be saved/read
mypath = 'C:\\Users\\AndyJ\\Desktop\\Carrosya\\Carros\\Batch\\celulares\\'
mylist = os.listdir(mypath) # list of files in 'mypath'


# First join between the two first csv files crawled (Table1 & Table2)
def first_join(): # It works with the crawled, generic and automatic batch files

    # Merge
    table2_mod = pd.merge(table1, table2, on=['a_id'], how='right', suffixes=('_old', '_new'))

    # Renamed scraped columns
    table2_mod.rename(columns={'b_link_new':'Link',
                               'c_producto_new':'Producto',
                               'd_marca_new':'Marca',
                               'e_modelo_new':'Modelo',
                               'f_version_new':'Version',
                               'g_precio_new':'Precio',
                               'h_KM_new':'KM',
                               'i_year_new':'Año',
                               'j_puertas_new':'Puertas',
                               'k_transmision_new':'Transmision',
                               'l_direccion_new':'Direccion',
                               'm_placa_new':'Placa',
                               'n_color_new':'Color',
                               'o_vendedor_new':'Vendedor',
                               'p_tel_contacto_new':'Tel_contacto',
                               'q_ubicacion_new':'Ubicacion',
                               'qa_info_adicional_new':'Info_adicional',
                               'qb_descripcion_new':'Descripcion',
                               'r_fecha_info_new':'Fecha_scraping'
                              },inplace=True)

    # Accumulated columns
    table2_mod['delta_precio'] = table2_mod['Precio'] - table2_mod['g_precio_old'] # Calculation
    table2_mod['delta_precio'].fillna(0, inplace=True) # Replace NAN with 0
    table2_mod['pct_delta_precio'] = table2_mod['Precio']/table2_mod['g_precio_old'] -1 # Calculation
    table2_mod['pct_delta_precio'].fillna(0, inplace=True) # # Replace NAN with 0
    table2_mod['flg_delta_precio'] = np.where(table2_mod['delta_precio'] != 0, 1,0) # Calculation
    table2_mod['Fecha_ini_monitor'] = table2_mod['r_fecha_info_old'].combine_first(table2_mod['Fecha_scraping']) # Calculation
    table2_mod['Ctd_dias_monitor'] = table2_mod['Fecha_scraping'] - table2_mod['Fecha_ini_monitor'] # Calculation
    table2_mod['Ctd_dias_monitor'] = table2_mod['Ctd_dias_monitor'] / np.timedelta64(1, 'D') # Remove 'days' word from results
    table2_mod['Ctd_dias_monitor'].fillna(0, inplace=True)
    table2_mod['Ctd_dias_monitor'] = table2_mod['Ctd_dias_monitor'].round()
    table2_mod['precio_inicial'] = table2_mod['g_precio_old'].combine_first(table2_mod['Precio']) # Calculation

    # Equivalents
    table2_mod['Acum_delta'] = table2_mod['delta_precio']
    table2_mod['Acum_pct_delta'] = table2_mod['pct_delta_precio']
    table2_mod['cambios_totales'] = table2_mod['flg_delta_precio']

    # NaN column for first join
    table2_mod['fecha_ult_mod'] = np.datetime64()

    # Selected columns
    table2_mod = table2_mod[['a_id',
                             'Link',
                             'Producto',
                             'Marca',
                             'Modelo',
                             'Version',
                             'Precio',
                             'KM',
                             'Año',
                             'Puertas',
                             'Transmision',
                             'Direccion',
                             'Placa',
                             'Color',
                             'Vendedor',
                             'Tel_contacto',
                             'Ubicacion',
                             'Info_adicional',
                             'Descripcion',
                             'Fecha_scraping',
                             'delta_precio',
                             'pct_delta_precio',
                             'Acum_delta',
                             'Acum_pct_delta',
                             'flg_delta_precio',
                             'cambios_totales',
                             'Ctd_dias_monitor',
                             'Fecha_ini_monitor',
                             'fecha_ult_mod',
                             'precio_inicial']]
    
    return table2_mod.to_csv(mypath + 'carros_mod_{0}.csv'.format(today_file), index=False)


# Nth join between csv files (Table_mod & TableN)
def nth_join(): # It needs the output file from 'first_join' function
    
    # Merge
    table3_mod = pd.merge(table2_mod, table3, on=['Id'], how='right', suffixes=('_old', '_new'))

    # Renamed scraped columns
    table3_mod.rename(columns={'b_link_new':'Link',
                               'c_producto_new':'Producto',
                               'd_marca_new':'Marca',
                               'e_modelo_new':'Modelo',
                               'f_version_new':'Version',
                               'g_precio_new':'Precio',
                               'h_KM_new':'KM',
                               'i_year_new':'Año',
                               'j_puertas_new':'Puertas',
                               'k_transmision_new':'Transmision',
                               'l_direccion_new':'Direccion',
                               'm_placa_new':'Placa',
                               'n_color_new':'Color',
                               'o_vendedor_new':'Vendedor',
                               'p_tel_contacto_new':'Tel_contacto',
                               'q_ubicacion_new':'Ubicacion',
                               'qa_info_adicional_new':'Info_adicional',
                               'qb_descripcion_new':'Descripcion',
                               'r_fecha_info_new':'Fecha_scraping'
                              },inplace=True)

    # Computed columns
    table3_mod['delta_precio'] = table3_mod['Precio'] - table3_mod['g_precio_old'] # Calculation
    table3_mod['delta_precio'].fillna(0, inplace=True) # Replace NAN with 0
    table3_mod['pct_delta_precio'] = table3_mod['Precio']/table3_mod['g_precio_old'] -1 # Calculation
    table3_mod['pct_delta_precio'].fillna(0, inplace=True) # Replace NAN with 0
    table3_mod['fecha_ult_mod'] = np.where(table3_mod['flg_delta_precio'] ==1, table3_mod['r_fecha_info_old'],table3_mod['fecha_ult_mod']) # Calculation
    table3_mod['flg_delta_precio'] = np.where(table3_mod['delta_precio'] != 0, 1,0) # Calculation
    table3_mod['Fecha_ini_monitor'] = table3_mod['Fecha_ini_monitor'].combine_first(table3_mod['Fecha_scraping']) # Calculation
    table3_mod['Ctd_dias_monitor'] = table3_mod['Fecha_scraping'] - table3_mod['Fecha_ini_monitor'] # Calculation
    table3_mod['Ctd_dias_monitor'] = table3_mod['Ctd_dias_monitor'] / np.timedelta64(1, 'D') # Remove 'days' word from results
    table3_mod['Ctd_dias_monitor'].fillna(0, inplace=True).astype(int) # Replace NAN with 0
    table3_mod['Ctd_dias_monitor'] = table3_mod['Ctd_dias_monitor'].round()
    table3_mod['precio_inicial'] = table3_mod['precio_inicial'].combine_first(table3_mod['Precio']) # Calculation
    table3_mod['Ctd_dias_ult_mod'] = table3_mod['Fecha_scraping'] - table3_mod['fecha_ult_mod'] # Calculation
    table3_mod['Ctd_dias_ult_mod'] = table3_mod['Ctd_dias_ult_mod'] / np.timedelta64(1, 'D') # Remove 'days' word from results
    table3_mod['Ctd_dias_ult_mod'].fillna(0, inplace=True) # Replace NAN with 0
    table3_mod['Ctd_dias_ult_mod'] = table3_mod['Ctd_dias_ult_mod'].round()

    # Accumulated columns
    table3_mod['Acum_delta'] = table3_mod['delta_precio'] + table3_mod['Acum_delta'] # Calculation
    table3_mod['Acum_delta'].fillna(0, inplace=True) # Replace NAN with 0
    table3_mod['Acum_pct_delta'] = table3_mod['pct_delta_precio'] + table3_mod['Acum_pct_delta']
    table3_mod['Acum_pct_delta'].fillna(0, inplace=True) # Replace NAN with 0
    table3_mod['cambios_totales'] = table3_mod['flg_delta_precio'] + table3_mod['cambios_totales']
    table3_mod['cambios_totales'].fillna(0, inplace=True) # Replace NAN with 0

    # Selected columns
    table3_mod = table3_mod[['a_id',
                             'Link',
                             'Producto',
                             'Marca',
                             'Modelo',
                             'Version',
                             'Precio',
                             'KM',
                             'Año',
                             'Puertas',
                             'Transmision',
                             'Direccion',
                             'Placa',
                             'Color',
                             'Vendedor',
                             'Tel_contacto',
                             'Ubicacion',
                             'Info_adicional',
                             'Descripcion',
                             'Fecha_scraping',
                             'delta_precio',
                             'pct_delta_precio',
                             'Acum_delta',
                             'Acum_pct_delta',
                             'flg_delta_precio',
                             'cambios_totales',
                             'Ctd_dias_monitor',
                             'Fecha_ini_monitor',
                             'fecha_ult_mod',
                             'precio_inicial']]
    
    return table3_mod.to_csv(mypath + 'carros_mod_{0}.csv'.format(today_file), index=False)


# Conditional for executing Function 
if not 'celulares_mod_{0}.csv'.format(yesterday_file) in mylist:
    
    table1 = pd.read_csv(mypath + 'carros_{0}.csv'.format(yesterday_file), 
                     parse_dates=['r_fecha_info'])

    table2 = pd.read_csv(mypath + 'carros_{0}.csv'.format(today_file), 
                     parse_dates=['r_fecha_info'])
    
    first_join() # calling the function
    
else:
    
    table2_mod = pd.read_csv(mypath + 'carros_mod_{0}.csv'.format(yesterday_file), 
                     parse_dates=['Fecha_scraping','Fecha_ini_monitor','fecha_ult_mod'])
    
    table3 = pd.read_csv(mypath + 'carros_{0}.csv'.format(today_file), 
                     parse_dates=['Fecha_scraping'])
    
    nth_join() # calling the function